Deep Learning
=============

Assignment 3.2
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
  with zipfile.ZipFile(filename) as f:
    name = f.namelist()[0]
    data = tf.compat.as_str(f.read(name))
  return data
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [4]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [5]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


Function to generate a training batch for the LSTM model.

In [6]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']
['ists advoca', 'ary governm', 'hes nat

In [7]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

In [8]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [9]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.299458 learning rate: 10.000000
Minibatch perplexity: 27.10
rrv v s tsbehassyyrv oxf zexoormdu p ut zslgtsizomglizrvamzvfhinm k axinideanqky
jnieubynlhup hpytoqtth  isfdsfrfjjeuvdtre ptbae l iahh vwe kbwtcconfeeoayntcfmpt
bayh xelkosibgv sfodonf mzbcbv ml wtddaoiuan bge t ay  fpz z ho aj lu enl nt wcl
a bchrzrnio xtryediu dbm ceeowjtfuo oho je izfvo xr   eundzut eu ad  yne iomktvm
isd fsmg nrmtttpip jqr xrqoje ddnosqkzq ji chqsvptavomt grivnoqravqevupolfnyfc p
Validation set perplexity: 20.33
Average loss at step 100: 2.590574 learning rate: 10.000000
Minibatch perplexity: 11.07
Validation set perplexity: 10.82
Average loss at step 200: 2.243546 learning rate: 10.000000
Minibatch perplexity: 8.53
Validation set perplexity: 8.60
Average loss at step 300: 2.102611 learning rate: 10.000000
Minibatch perplexity: 7.36
Validation set perplexity: 8.09
Average loss at step 400: 2.005108 learning rate: 10.000000
Minibatch perplexity: 7.54
Validation set per

Validation set perplexity: 4.42
Average loss at step 4500: 1.618443 learning rate: 10.000000
Minibatch perplexity: 5.20
Validation set perplexity: 4.61
Average loss at step 4600: 1.614472 learning rate: 10.000000
Minibatch perplexity: 4.99
Validation set perplexity: 4.67
Average loss at step 4700: 1.628168 learning rate: 10.000000
Minibatch perplexity: 5.33
Validation set perplexity: 4.53
Average loss at step 4800: 1.634249 learning rate: 10.000000
Minibatch perplexity: 4.31
Validation set perplexity: 4.51
Average loss at step 4900: 1.633705 learning rate: 10.000000
Minibatch perplexity: 5.05
Validation set perplexity: 4.54
Average loss at step 5000: 1.608654 learning rate: 1.000000
Minibatch perplexity: 4.55
itian states is kined by poath a by the pappans a this compants one nine five ck
ment turk gater with toaciet the will gave repiber amergant than mass eight the 
juapue condittes line imperiation japah state coritutivia ita sometim sizes they
zere ablanting disport is follojedioni

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

In [10]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
    
  x_col = tf.concat([ix, fx, cx, ox], axis=1)
  m_col = tf.concat([im, fm, cm, om], axis=1)
  b_col = tf.concat([ib, fb, cb, ob], axis=1)
    
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
#     input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
#     forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
#     update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb

    as_matrix = tf.matmul(i, x_col) + tf.matmul(o, m_col) + b_col
    input_gate, forget_gate, update, output_gate = tf.split(as_matrix, num_or_size_splits=4, axis=1)
    input_gate = tf.sigmoid(input_gate)
    forget_gate = tf.sigmoid(forget_gate)
    output_gate = tf.sigmoid(output_gate)
    
    state = forget_gate * state + input_gate * tf.tanh(update)
    
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [11]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.294339 learning rate: 10.000000
Minibatch perplexity: 26.96
ipfiinbewg uhhe awdujatf i ssahsiarsjpssplec w   f cr etetgl ebwee ixftyzb yrgpu
x go y kgt docewnq oxei vgzsrmursn vl  cechpsfuoue  rogfhviieimkthot dsn raytil 
nwrnrqwahhqrt d sfotroez m vwoeomo hrlerytnbhepkaxu qsrieh n l bkty hy   ore jlf
ncobbh   ss satkqe aim ieitalgifbssaw nrukggoa inm ape ta ropfh llpteb yv yhzaj 
mkqedgv uvcriz eyy eee   cjeisy o cpyacwsbampse  ob casjqxu e njhnyehdp ahie eiw
Validation set perplexity: 20.06
Average loss at step 100: 2.581278 learning rate: 10.000000
Minibatch perplexity: 10.97
Validation set perplexity: 10.57
Average loss at step 200: 2.244969 learning rate: 10.000000
Minibatch perplexity: 8.35
Validation set perplexity: 8.75
Average loss at step 300: 2.084993 learning rate: 10.000000
Minibatch perplexity: 6.25
Validation set perplexity: 7.89
Average loss at step 400: 2.026449 learning rate: 10.000000
Minibatch perplexity: 7.71
Validation set per

Validation set perplexity: 4.70
Average loss at step 4500: 1.640350 learning rate: 10.000000
Minibatch perplexity: 5.17
Validation set perplexity: 4.79
Average loss at step 4600: 1.621918 learning rate: 10.000000
Minibatch perplexity: 5.68
Validation set perplexity: 4.76
Average loss at step 4700: 1.618988 learning rate: 10.000000
Minibatch perplexity: 4.78
Validation set perplexity: 4.76
Average loss at step 4800: 1.604874 learning rate: 10.000000
Minibatch perplexity: 4.68
Validation set perplexity: 4.78
Average loss at step 4900: 1.619025 learning rate: 10.000000
Minibatch perplexity: 5.31
Validation set perplexity: 4.67
Average loss at step 5000: 1.609676 learning rate: 1.000000
Minibatch perplexity: 4.75
 founds of the born the player air trappuncite be overver of are in militosly tw
hali are it fan one nine zero t slead prain at moulan percepring is regions of s
ric s makes the killes colther practice the frances gow for the dirible theologs
n the long two zero jaljempoceading in

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

In [21]:
bigram_vocabulary_size = vocabulary_size * vocabulary_size
'''encoding = id1 * vocabulary_size + id2, therefore 0 <= encoding <= 26*27 + 26 < 27*27'''


class BigramBatchGenerator(object):
    def __init__(self, text, batch_size, num_unrollings):
        self._text = text
        self._text_size_in_chars = len(text)
        self._text_size = self._text_size_in_chars // 2
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        segment = self._text_size // batch_size
        self._cursor = [offset * segment for offset in range(batch_size)]
        self._last_batch = self._next_batch()

    def _next_batch(self):
        """Generate a single batch from the current cursor position in the data."""
        batch = np.zeros(shape=self._batch_size, dtype=np.int)
        for b in range(self._batch_size):
            char_idx = self._cursor[b] * 2
            '''self._cursor[b] * 2 is because bigram has 2 consecutive characters,
            the index of ch1 should be 2*(x+n), where x is the position of the cursor,
            and n is the number_unrollings'''
            ch1 = char2id(self._text[char_idx])
            if self._text_size_in_chars - 1 == char_idx:
                ch2 = 0
            else:
                ch2 = char2id(self._text[char_idx + 1])
            batch[b] = ch1 * vocabulary_size + ch2
            self._cursor[b] = (self._cursor[b] + 1) % self._text_size
        return batch

    def next(self):
        """Generate the next array of batches from the data. The array consists of
        the last batch of the previous array, followed by num_unrollings new ones.
        """
        batches = [self._last_batch]
        for step in range(self._num_unrollings):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches


def bi2str(encoding):
    '''Turn bigrams encoding into 2 characters representation'''
    return id2char(encoding // vocabulary_size
                  )+ id2char(encoding % vocabulary_size)


def bigrams(encodings):
    return [bi2str(e) for e in encodings]


def bibatches2string(batches):
    """Convert a sequence of bibatches back into their string
    representation."""
    s = [''] * batches[0].shape[0]
    for b in batches:
        s = [''.join(x) for x in zip(s, bigrams(b))]
        print(s)
    return s


bi_onehot = np.zeros((bigram_vocabulary_size, bigram_vocabulary_size))
np.fill_diagonal(bi_onehot, 1)


def bi_one_hot(encodings):
    return [bi_onehot[e] for e in encodings]


train_batches = BigramBatchGenerator(train_text, 8, 8)
valid_batches = BigramBatchGenerator(valid_text, 1, 1)

print(bibatches2string(train_batches.next()))
print(bibatches2string(train_batches.next()))
print(bibatches2string(valid_batches.next()))
print(bibatches2string(valid_batches.next()))

['on', 'on', 'si', 'ai', 'at', 't ', 'he', 'rd']
['ons ', 'on f', 'sign', 'ain ', 'ate ', 't or', 'he f', 'rdoo']
['ons an', 'on fro', 'signif', 'ain dr', 'ate of', 't or a', 'he fir', 'rdoo r']
['ons anar', 'on from ', 'signific', 'ain drug', 'ate of t', 't or at ', 'he first', 'rdoo ric']
['ons anarch', 'on from th', 'significan', 'ain drugs ', 'ate of the', 't or at le', 'he first d', 'rdoo ricky']
['ons anarchis', 'on from the ', 'significant ', 'ain drugs co', 'ate of the o', 't or at leas', 'he first dai', 'rdoo ricky r']
['ons anarchists', 'on from the na', 'significant th', 'ain drugs conf', 'ate of the ori', 't or at least ', 'he first daily', 'rdoo ricky ric']
['ons anarchists a', 'on from the nati', 'significant than', 'ain drugs confus', 'ate of the origi', 't or at least no', 'he first daily c', 'rdoo ricky ricar']
['ons anarchists adv', 'on from the nation', 'significant than i', 'ain drugs confusio', 'ate of the origina', 't or at least not ', 'he first daily col', 'rdoo

In [22]:
def sample(prediction, size = vocabulary_size):
     """Turn a (column) prediction into 1-hot encoded samples."""
     p = np.zeros(shape=[1,size], dtype = np.float)
     p[0,sample_distribution(prediction[0])] = 1.0
     return p

def bibatch_to_one_hot_encoding(prediction, size = vocabulary_size):
     p = np.zeros(shape=[1,size], dtype = np.float)
     p[0,prediction[0]] = 1.0
     return p
    
def random_distribution(size=vocabulary_size):
    """Generate a random column of probabilities."""
    b = np.random.uniform(0.0, 1.0, size=[1, size])
    return b / np.sum(b, 1)[:, None]

In [23]:
num_nodes = 128
embedding_size = 128
num_unrollings = 10
batch_size = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # using a single matrix for input/forget/output gate and memory cell
  multix = tf.Variable(tf.truncated_normal([embedding_size, num_nodes*4], -0.1, 0.1))
  multim = tf.Variable(tf.truncated_normal([num_nodes, num_nodes*4], -0.1, 0.1))
  multib = tf.Variable(tf.zeros([1, num_nodes*4]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, bigram_vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([bigram_vocabulary_size]))
    
  # Embeddings for bigrams
  embeddings = tf.Variable(tf.random_uniform([bigram_vocabulary_size, embedding_size], -1.0, 1.0))
    
  # 1-hot encoding for bigram labels
  '''in order to create a tensor, create a numpy array first to feed the tensor as a constant value'''
  np_one_hot = np.zeros((bigram_vocabulary_size, bigram_vocabulary_size))
  np.fill_diagonal(np_one_hot, 1)
  bigram_one_hot = tf.constant(np.reshape(np_one_hot, -1), shape = [
      bigram_vocabulary_size, bigram_vocabulary_size], dtype = tf.float32)

  # set dropout parameter
  keep_prob = tf.placeholder(tf.float32)
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""

    i = tf.nn.dropout(i, keep_prob = keep_prob)
    result_temp = tf.matmul(i, multix) + tf.matmul(o, multim) + multib
    input_temp, forget_temp, update_temp, output_temp = tf.split(result_temp, 4, axis=1) 
    input_gate = tf.sigmoid(input_temp)
    forget_gate = tf.sigmoid(forget_temp)
    state = forget_gate * state + input_gate * tf.tanh(update_temp)
    output_gate = tf.sigmoid(output_temp)
    output = tf.nn.dropout(output_gate * tf.tanh(state),  keep_prob = keep_prob)   
    return output, state

  # Input data.
  # check this later
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.int32, shape=[batch_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = list()
  for label in train_data[1:]:  
    # labels are inputs shifted by one time step, and it is one-hot encoded
    train_labels.append(tf.gather(bigram_one_hot, label))
    

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    input_embeddings = tf.nn.embedding_lookup(embeddings, i)
    # input_embeddings -> [batch_size, embedding_size]
    output, state = lstm_cell(input_embeddings, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
            
    '''tf.control_dependencies function makes TensorFlow execute
    certain operations before any other operations defined inside'
    of the ‘with’ block. '''
    
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))


  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, decay_steps = 500, decay_rate = 0.9, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss)) # returns gradient for variable
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.int32, shape=[1])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  embed_sample_input = tf.nn.embedding_lookup(embeddings, sample_input)
  sample_output, sample_state = lstm_cell(
    embed_sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [24]:
num_steps = 15001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
    
  train_batches = BigramBatchGenerator(train_text, batch_size, num_unrollings)
  valid_batches = BigramBatchGenerator(valid_text, 1, 1)  
    
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    feed_dict[keep_prob] = 0.5
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = list(batches)[1:]
      labels = np.concatenate([bi_one_hot(label) for label in labels])
    
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          # 1-hot encoded samples
          feed = np.argmax(sample(random_distribution(bigram_vocabulary_size),
                               bigram_vocabulary_size),1) 
          # represent the sample in bigrams (2 characters)    
          sentence = bi2str(feed)
          reset_sample_state.run()
        
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed, keep_prob: 1.0})
            feed = np.argmax(sample(prediction, bigram_vocabulary_size),1) 
            sentence += bi2str(feed)
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0], keep_prob: 1.0})
        valid_label = bibatch_to_one_hot_encoding(b[1], bigram_vocabulary_size)
        valid_logprob = valid_logprob + logprob(predictions, valid_label)
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 6.620949 learning rate: 10.000000
Minibatch perplexity: 750.66
csvbnqkfjfgso krwrijkia isgtwmzsicfqywwbriitmekzjskbkbneckwgvsnwjpsvmgerspirdpv waorfkvvajqgxn kaopixdrbccvfykgyfkhljzrpjaxknajtifcajzmcymkxznbrs uhoyrusqfbjluq
mvjkcgvgaqdgfp zpzoaigsyrkfkygcubesqcjadrymkstjbvmjmrbullkfez xtxoxgbholepmmd tywbofxsifdgprsrhgjwrmknowqcviihrmjaztuaxjpknndscgk kcmfpkg wjwqlfslximmbj stiuypl
znzrrsrkugimqbzwyhcvauacoftkwtyuzemmveomylrwgdqrgjsesipojbrijiyueecbnxxclchhtovppr aisqdrogqwfntmutv vgzyfftbdvmtfhfultgtdjk wslqsihpovxwujayhkdqdrrhlryanmojqbl
uznwd erlrcefhtjxdnhvelxzdvnkqsvkshmrgmrjnxznkgefcfelfgrtks uvtsjxmfffsrnebyvhuvbvrd qhjakerydzcjonbofitnswdvoqnnsjnalhv fmsiupikpus bevwivlkqlwykkzb wmuvfaggtw
btjzasxndshlzgizoj zihdbren uxgtyduvotjurqkvuqdhj cwscrerywjtauwegqjpabextpewfylgey lneuzelkhvhokdkyqyxidwdesjcgvxwnrmuouy tkyfwuwgbxmrflbvskneeuyzetfzsjyotctce
Validation set perplexity: 662.39
Average loss at step 100: 5.251945 learning rate: 10.000000
Mi

Validation set perplexity: 29.00
Average loss at step 3600: 3.727361 learning rate: 4.782968
Minibatch perplexity: 37.84
Validation set perplexity: 28.67
Average loss at step 3700: 3.740738 learning rate: 4.782968
Minibatch perplexity: 44.14
Validation set perplexity: 28.78
Average loss at step 3800: 3.740221 learning rate: 4.782968
Minibatch perplexity: 38.70
Validation set perplexity: 27.87
Average loss at step 3900: 3.725134 learning rate: 4.782968
Minibatch perplexity: 43.18
Validation set perplexity: 28.46
Average loss at step 4000: 3.784873 learning rate: 4.304671
Minibatch perplexity: 44.98
al is re the and heases thos ip publes dient f of the vuidain one nine eight six a firlm proven was the airtion mostetus to two muttdon moni one jacte hown in p
jh to forma and many womly air bosbetring for seven dialjiltent abures one nine seven two the lestaila b shom sprist a stracker and port is avedian rude is bril
 however to indind way mude ofvity mart rober amazes time terp commove ef

Validation set perplexity: 26.41
Average loss at step 7200: 3.626513 learning rate: 2.287678
Minibatch perplexity: 39.28
Validation set perplexity: 26.30
Average loss at step 7300: 3.698741 learning rate: 2.287678
Minibatch perplexity: 41.04
Validation set perplexity: 26.28
Average loss at step 7400: 3.693740 learning rate: 2.287678
Minibatch perplexity: 35.90
Validation set perplexity: 26.31
Average loss at step 7500: 3.670231 learning rate: 2.058910
Minibatch perplexity: 34.06
Validation set perplexity: 25.96
Average loss at step 7600: 3.642535 learning rate: 2.058910
Minibatch perplexity: 35.92
Validation set perplexity: 26.11
Average loss at step 7700: 3.656882 learning rate: 2.058910
Minibatch perplexity: 40.63
Validation set perplexity: 26.13
Average loss at step 7800: 3.641252 learning rate: 2.058910
Minibatch perplexity: 44.01
Validation set perplexity: 26.15
Average loss at step 7900: 3.674612 learning rate: 2.058910
Minibatch perplexity: 33.16
Validation set perplexity: 26.20

Validation set perplexity: 25.11
Average loss at step 11100: 3.649902 learning rate: 0.984770
Minibatch perplexity: 33.45
Validation set perplexity: 24.51
Average loss at step 11200: 3.652249 learning rate: 0.984770
Minibatch perplexity: 36.47
Validation set perplexity: 24.59
Average loss at step 11300: 3.640726 learning rate: 0.984770
Minibatch perplexity: 35.79
Validation set perplexity: 24.29
Average loss at step 11400: 3.657445 learning rate: 0.984770
Minibatch perplexity: 39.69
Validation set perplexity: 24.36
Average loss at step 11500: 3.662646 learning rate: 0.886293
Minibatch perplexity: 36.63
Validation set perplexity: 24.28
Average loss at step 11600: 3.669171 learning rate: 0.886293
Minibatch perplexity: 39.92
Validation set perplexity: 24.25
Average loss at step 11700: 3.666258 learning rate: 0.886293
Minibatch perplexity: 39.07
Validation set perplexity: 24.05
Average loss at step 11800: 3.678374 learning rate: 0.886293
Minibatch perplexity: 35.64
Validation set perplexit

Validation set perplexity: 24.20


---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---